<a href="https://colab.research.google.com/github/okuokuch/pokemon_home/blob/main/pokemon_home.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pokemon HOMEのシーズン情報取得

Githubからファイルを取得

In [1]:
!git clone https://github.com/okuokuch/pokemon_home.git

Cloning into 'pokemon_home'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 26 (delta 8), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (26/26), done.


In [15]:
import json
import pandas as pd
import ast
df_poke = pd.read_csv("./pokemon_home/poke_id.csv", encoding="UTF-8")
with open("./pokemon_home/item.json","r",encoding="UTF-8") as f:
  data = f.read()
  item_json = ast.literal_eval(data)
with open("./pokemon_home/move.json","r",encoding="UTF-8") as f:
  data = f.read()
  move_json = ast.literal_eval(data)
with open("./pokemon_home/seikaku.json","r",encoding="UTF-8") as f:
  data = f.read()
  seikaku_json = ast.literal_eval(data)
with open("./pokemon_home/tokusei.json","r",encoding="UTF-8") as f:
  data = f.read()
  tokusei_json = ast.literal_eval(data)

In [4]:
import requests

#データ取得のためのダミー設定値。スマホからアクセスしているような設定。
headers = {
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'countrycode': '304',
    'authorization': 'Bearer',
    'langcode': '1',
    'user-agent': 'Mozilla/5.0 (Linux; Android 8.0; Pixel 2 Build/OPD3.170816.012) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Mobile Safari/537.36',
    'content-type': 'application/json'
}

data = '{"soft":"Sw"}'
#リクエストを出してシーズン情報を取得
response_base = requests.post('https://api.battle.pokemon-home.com/cbd/competition/rankmatch/list', headers=headers, data=data)
info_all = json.loads(response_base.text)

# 取り出したい情報への記入方法
seasonの中に取り出したいシーズン番号を記入してください。  
ruleには0か1を記入してください。(0ならシングルで1ならダブルの情報を取得します)

In [6]:
#@title 取り出したい情報 { display-mode: "both" }
#seasonには数字を入力して下し
#ruleにはシングルなら1、ダブルなら0を入力してください。
season = "30" #@param {type:"string"}
rule = 0 #@param {type:"integer"}


In [7]:
#指定したシーズン情報を取り出すためのリクエストパラメータ取得
info_season = info_all['list'][season]

for content in info_season:
  if info_season[content]['rule'] == rule:
    params = {'id':content, 'rst':str(info_season[content]['rst']), 'ts1':str(info_season[content]['ts1']), 'ts2':str(info_season[content]['ts2'])}

In [10]:
pokemon_json ={}
for x in range(1,6):
    adress = "https://resource.pokemon-home.com/battledata/ranking/"+params["id"]+"/"+params["rst"]+"/"+params["ts2"]+"/pdetail-"+str(x)
    headers = {
        'user-agent': 'Mozilla/5.0 (Linux; Android 8.0; Pixel 2 Build/OPD3.170816.012) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Mobile Safari/537.36',
        'content-type': 'application/json',
    }

    response = requests.get(adress, headers=headers)
    json_read= json.loads(response.text)
    pokemon_json.update(json_read)

In [52]:
from tqdm import tqdm
df_poke_item = pd.DataFrame()
df_poke_nature = pd.DataFrame()
df_poke_ability = pd.DataFrame()
df_poke_move = pd.DataFrame()


for poke in tqdm(pokemon_json):
  for region in pokemon_json[str(poke)]:
    #ポケモン名を取得
    name = df_poke[(df_poke['id'] == int(poke)) & (df_poke['リージョン'] == int(region))]['ポケモン名'].values[0]
    temoti = pokemon_json[str(poke)][region]['temoti']

    list_motimono = [name]
    list_seikaku = [name]
    list_tokusei = [name]
    list_waza = [name]

    #もちもので展開しlistに格納。
    for info_motimono in temoti['motimono']:
      motimono = item_json[int(info_motimono['id'])]
      list_motimono.append(motimono)
      list_motimono.append(info_motimono['val'])    
    #性格で展開しlistに格納。
    for info_seikaku in temoti['seikaku']:
      seikaku = seikaku_json[int(info_seikaku['id'])]
      list_seikaku.append(seikaku)
      list_seikaku.append(info_seikaku['val'])
    #特性で展開しlistに格納。
    for info_tokusei in temoti['tokusei']:
      tokusei = tokusei_json[int(info_tokusei['id'])]
      list_tokusei.append(seikaku)
      list_tokusei.append(info_tokusei['val'])
    #技名で展開しlistに格納。
    for info_waza in temoti['waza']:
      waza = move_json[int(info_waza['id'])]
      list_waza.append(waza)
      list_waza.append(info_waza['val'])
    df_poke_item = df_poke_item.append([list_motimono])
    df_poke_nature = df_poke_nature.append([list_seikaku])
    df_poke_ability = df_poke_ability.append([list_tokusei])
    df_poke_move = df_poke_move.append([list_waza])

100%|██████████| 492/492 [00:07<00:00, 69.67it/s]


In [69]:
#各データフレームはポケモン名、要素1、割合1、要素2、割合2、、、となるのでそれに沿ってカラム名付与
column_item = ['ポケモン名']
column_nature = ['ポケモン名']
column_ability = ['ポケモン名']
column_move = ['ポケモン名']

column_item += ['アイテム', '割合']*int((len(df_poke_item.columns)-1)/2)
column_nature += ['アイテム', '割合']*int((len(df_poke_nature.columns)-1)/2)
column_ability += ['アイテム', '割合']*int((len(df_poke_ability.columns)-1)/2)
column_move += ['アイテム', '割合']*int((len(df_poke_move.columns)-1)/2)

df_poke_item.columns = column_item
df_poke_nature.columns = column_nature
df_poke_ability.columns = column_ability
df_poke_move.columns = column_move

# Google Driveにcsvファイルを書き出し。
Google Driveをマウントして、そこにcsvファイルを書き出しする。

In [75]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [78]:
#必要に応じて"./drive/MyDrive/poke_home_item.csv"の部分を変更してください。この設定だと、おそらく接続したGoogle Driveのトップページに保存されます。
#もしdataというフォルダの中に保存したい場合は、"./drive/MyDrive/data/poke_home_item.csv"とすればよいはずです。
df_poke_item.to_csv("./drive/MyDrive/poke_home_item.csv",encoding="shift-jis")
df_poke_nature.to_csv("./drive/MyDrive/poke_home_nature.csv",encoding="shift-jis")
df_poke_ability.to_csv("./drive/MyDrive/poke_home_ability.csv",encoding="shift-jis")
df_poke_move.to_csv("./drive/MyDrive/poke_home_move.csv",encoding="shift-jis")